# AutoML: 「最適な」画像オブジェクト検出モデルを学習する

**目的** - AutoML の画像オブジェクト検出ジョブの設定と実行方法について説明します。これは AutoML がサポートする9つの ML タスクのうちの1つです。他の ML タスクには、「予測」、「分類」、「画像オブジェクト検出」、「NLP テキスト分類」などがあります。

このノートブックでは、小さなデータセットを使って AutoML によりモデルを学習し、モデルの性能を最適化するためにモデルのハイパーパラメータを調整する方法を示し、推論シナリオで使用するモデルをデプロイします。

**前提条件** - このハンズオンでは、以下が前提条件となります:
- Machine Learning の基礎知識
- 利用可能な Azure subscription. [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- Azure ML workspace [Check this notebook for creating a workspace](../../../resources/workspace/workspace.ipynb) 
- Compute Cluster [Check this notebook to create a compute cluster](../../../resources/compute/compute.ipynb)
- Python 実行環境
- Azure Machine Learning Python SDK v2 - [install instructions](../../../README.md)

**学習のゴール** - このハンズオンを完了すると、以下を学習できます:
- Python SDK から AML workspacce への接続方法
- 'image_object_detection()' 関数を使って `AutoML Image Object Detection Training Job` を実行する方法
- Azure Machine Learning のコンピュートで AutoML 機能を使ってモデルを構築する方法
- 学習済みモデルと推論スコアの取得 

# 1. Azure Machine Learning Workspace への接続

[Workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) は Azure Machine Learning における最上位のリソースで、チームにおける機械学習成果物を作成し、関連作業をグループ化して管理する場所です。 実験、ジョブ、データセット、モデル、コンポーネント、推論エンドポイントなどの機械学習開発における成果物を統合し、共同作業を効率化することが可能です。
このセクションでは、ジョブを実行する Workspace に接続する。

## 1.1. Python ライブラリの import

In [ ]:
# 必要なライブラリのインポート
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

from azure.ai.ml.automl import SearchSpace, ObjectDetectionPrimaryMetrics
from azure.ai.ml.sweep import (
    Choice,
    Uniform,
    BanditPolicy,
)

from azure.ai.ml import automl

## 1.2. Workspace の詳細を設定し接続する

ワークスペースに接続するには、パラメータ情報（サブスクリプション、リソースグループ、ワークスペース名）が必要です。これらの情報を `azure.ai.ml` の `MLClient` で使用して、必要な Azure Machine Learning ワークスペースへ接続して各種操作を可能にします。このチュートリアルでは、デフォルトの [default azure authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) を使用します。

パラメータ情報（サブスクリプション、リソースグループ、ワークスペース名）を直接指定して接続することも可能です。パラメータについては、ワークスペースの右上のタブから確認することが可能です。

In [ ]:
credential = DefaultAzureCredential()
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your AML workspace
    subscription_id = "<SUBSCRIPTION_ID>"
    resource_group = "<RESOURCE_GROUP>"
    workspace = "<AML_WORKSPACE_NAME>"
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)

# 2. MLTable と入力トレーニングデータ

AutoML を用いてコンピュータビジョンのモデルを生成するには、モデルトレーニングの入力としてラベル付き画像データを MLTable の形式で提供する必要があります。MLTable は、様々なデータソースからのデータを統一的に扱うための抽象化された論理的なデータフォーマットを定義することが可能です。


ラベル付きトレーニングデータは JSONL 形式で MLTable を作成することができます。もしラベル付きトレーニングデータが異なる形式（例えば、pascal VOCやCOCOなど）である場合、まずそれをJSONL形式に変換するための変換スクリプトを使用し、その後で MLTable を作成できます。または、Azure Machine Learning のデータラベリングツールを使用して手動で画像にラベルを付け、ラベル付けされたデータをエクスポートして AutoML モデルのトレーニングに使用することもできます。

このノートブックでは、「Fridge Objects」とのデータセットを使用しています。これは、異なるシーンで撮影された4つのラベル（缶、紙パック、ミルクボトル、水ボトル）の飲料容器の写真128枚から構成されています。
このノートブックのすべての画像は[このリポジトリ](https://github.com/microsoft/computervision-recipes)にホストされており、[MIT license](https://github.com/microsoft/computervision-recipes/blob/master/LICENSE)の下で利用可能です。

## 2.1. データのダウンロード
まず、データをローカルにダウンロードして解凍します。デフォルトでは、データは現在のディレクトリ内の./dataフォルダにダウンロードされます。

データを別の場所にダウンロードしたい場合は、次のセルのdataset_parent_dir = ...を更新してください。

In [ ]:
import os
import urllib
from zipfile import ZipFile

# Change to a different location if you prefer
dataset_parent_dir = "./data"

# create data folder if it doesnt exist.
os.makedirs(dataset_parent_dir, exist_ok=True)

# download data
download_url = "https://cvbp-secondary.z19.web.core.windows.net/datasets/object_detection/odFridgeObjects.zip"

# Extract current dataset name from dataset url
dataset_name = os.path.split(download_url)[-1].split(".")[0]
# Get dataset path for later use
dataset_dir = os.path.join(dataset_parent_dir, dataset_name)

# Get the data zip file path
data_file = os.path.join(dataset_parent_dir, f"{dataset_name}.zip")

# Download the dataset
urllib.request.urlretrieve(download_url, filename=data_file)

# extract files
with ZipFile(data_file, "r") as zip:
    print("extracting files...")
    zip.extractall(path=dataset_parent_dir)
    print("done")
# delete zip file
os.remove(data_file)

データセット内のサンプル画像データを確認

In [ ]:
from IPython.display import Image

sample_image = os.path.join(dataset_dir, "images", "31.jpg")
Image(filename=sample_image)

## 2.2. AML データアセット（URIフォルダ）としてデータストアに画像をアップロードする。

Azure ML でモデルトレーニングにデータを利用するために、ワークスペースのデフォルト Blob Storage にデータをアップロードします。

[Check this notebook for AML data asset example](../../../assets/data/data.ipynb)

In [ ]:
# Uploading image files by creating a 'data asset URI FOLDER':

from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.ai.ml import Input

my_data = Data(
    path=dataset_dir,
    type=AssetTypes.URI_FOLDER,
    description="Fridge-items images Object detection",
    name="fridge-items-images-object-detection",
)

uri_folder_data_asset = ml_client.data.create_or_update(my_data)

print(uri_folder_data_asset)
print("")
print("Path to folder in Blob Storage:")
print(uri_folder_data_asset.path)

## 2.3. ダウンロードしたデータを JSONL 形式に変換

この例では、冷蔵庫オブジェクトのデータセットが Pascal VOC 形式でアノテーションされており、各画像にはxmlファイルが対応しています。各xmlファイルには、対応する画像ファイルの場所と、バウンディングボックスおよびオブジェクトラベルに関する情報が含まれています。

このデータを AzureML MLTable で使用するためには、特定の JSONL 形式に変換する必要があります。以下のスクリプトは、対応する MLTable フォルダ内に2つの .jsonl ファイル（1つはトレーニング用、もう1つはバリデーション用）を作成します。トレーニング/バリデーションの比率は、4:1 とします。AutoML で画像分類タスクに使用される jsonl ファイルの詳細については、[ドキュメント](https://learn.microsoft.com/en-us/azure/machine-learning/reference-automl-images-schema#object-detection)を参照してください。

## JSONLファイルを生成する

JSONL 変換コードは pycocotools と simplification パッケージを必要とします。

In [ ]:
!/anaconda/envs/azureml_py310_sdkv2/bin/pip install pycocotools
!/anaconda/envs/azureml_py310_sdkv2/bin/pip install simplification
!/anaconda/envs/azureml_py310_sdkv2/bin/pip install scikit-image

In [ ]:
!/anaconda/envs/azureml_py310_sdkv2/bin/pip install torch==1.12.0+cu102 torchvision==0.13.0+cu102 torchaudio==0.12.0 --extra-index-url https://download.pytorch.org/whl/cu102

In [ ]:
import sys

sys.path.insert(0, "../jsonl-conversion/")
from base_jsonl_converter import write_json_lines
from voc_jsonl_converter import VOCJSONLConverter

base_url = os.path.join(uri_folder_data_asset.path, "images/")
converter = VOCJSONLConverter(base_url, os.path.join(dataset_dir, "annotations"))
jsonl_annotations = os.path.join(dataset_dir, "annotations_voc.jsonl")
write_json_lines(converter, jsonl_annotations)

## トレーニング/バリデーションにデータを分割

In [ ]:
import os

# We'll copy each JSONL file within its related MLTable folder
training_mltable_path = os.path.join(dataset_parent_dir, "training-mltable-folder")
validation_mltable_path = os.path.join(dataset_parent_dir, "validation-mltable-folder")

# First, let's create the folders if they don't exist
os.makedirs(training_mltable_path, exist_ok=True)
os.makedirs(validation_mltable_path, exist_ok=True)

train_validation_ratio = 5

# Path to the training and validation files
train_annotations_file = os.path.join(training_mltable_path, "train_annotations.jsonl")
validation_annotations_file = os.path.join(
    validation_mltable_path, "validation_annotations.jsonl"
)

with open(jsonl_annotations, "r") as annot_f:
    json_lines = annot_f.readlines()

index = 0
with open(train_annotations_file, "w") as train_f:
    with open(validation_annotations_file, "w") as validation_f:
        for json_line in json_lines:
            if index % train_validation_ratio == 0:
                # validation annotation
                validation_f.write(json_line)
            else:
                # train annotation
                train_f.write(json_line)
            index += 1

## 2.4. アノテーションファイルを COCO からJSONL に変換する
COCO 形式のデータセットを使ってみたい場合は、以下のスクリプトで`jsonl`形式に変換することができます。odFridgeObjects_coco.json ファイルは `odFridgeObjects` データセットのアノテーション情報から構成されています。

In [ ]:
import sys

sys.path.insert(0, "../jsonl-conversion/")
from base_jsonl_converter import write_json_lines
from coco_jsonl_converter import COCOJSONLConverter

base_url = os.path.join(uri_folder_data_asset.path, "images/")
print(base_url)
converter = COCOJSONLConverter(base_url, "./odFridgeObjects_coco.json")
jsonl_annotations = os.path.join(dataset_dir, "annotations_coco.jsonl")
write_json_lines(converter, jsonl_annotations)

### バウンディング・ボックスの可視化
以下の[チュートリアル](https://docs.microsoft.com/en-us/azure/machine-learning/tutorial-auto-train-image-models#visualize-data)の "データの可視化 "セクションを参照して、トレーニングを開始する前に、グランドトゥルースのバウンディングボックスを簡単に可視化する方法を確認してください。

## 2.5. MLTable 作成
上記で作成したjsonlファイルを使ってMLTableデータ入力を作成します。

このノートブック以外のジョブのために独自の MLTable アセットを作成するドキュメントについては、以下のリソースを参照してください。
- [MLTable YAML Schema](https://learn.microsoft.com/en-us/azure/machine-learning/reference-yaml-mltable) - covers how to write MLTable YAML, which is required for each MLTable asset.
- [Create MLTable data asset](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-create-data-assets?tabs=Python-SDK#create-a-mltable-data-asset) - covers how to create MLTable data asset. 

In [ ]:
def create_ml_table_file(filename):
    """Create ML Table definition"""

    return (
        "paths:\n"
        "  - file: ./{0}\n"
        "transformations:\n"
        "  - read_json_lines:\n"
        "        encoding: utf8\n"
        "        invalid_lines: error\n"
        "        include_path_column: false\n"
        "  - convert_column_types:\n"
        "      - columns: image_url\n"
        "        column_type: stream_info"
    ).format(filename)


def save_ml_table_file(output_path, mltable_file_contents):
    with open(os.path.join(output_path, "MLTable"), "w") as f:
        f.write(mltable_file_contents)


# Create and save train mltable
train_mltable_file_contents = create_ml_table_file(
    os.path.basename(train_annotations_file)
)
save_ml_table_file(training_mltable_path, train_mltable_file_contents)

# Save train and validation mltable
validation_mltable_file_contents = create_ml_table_file(
    os.path.basename(validation_annotations_file)
)
save_ml_table_file(validation_mltable_path, validation_mltable_file_contents)

In [ ]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes

# Training MLTable defined locally, with local data to be uploaded
my_training_data_input = Input(type=AssetTypes.MLTABLE, path=training_mltable_path)

# Validation MLTable defined locally, with local data to be uploaded
my_validation_data_input = Input(type=AssetTypes.MLTABLE, path=validation_mltable_path)

# WITH REMOTE PATH: If available already in the cloud/workspace-blob-store
# my_training_data_input = Input(type=AssetTypes.MLTABLE, path="azureml://datastores/workspaceblobstore/paths/vision-classification/train")
# my_validation_data_input = Input(type=AssetTypes.MLTABLE, path="azureml://datastores/workspaceblobstore/paths/vision-classification/valid")

V1 SDK を使用して作成した TabularDataset からデータ入力を作成するには、`type`に `AssetTypes.MLTABLE`、`mode`に `InputOutputModes.DIRECT`、`path`に `azureml:<tabulardataset_name>:<version>` という形式で指定します。

In [ ]:
"""
# Training MLTable with v1 TabularDataset
my_training_data_input = Input(
    type=AssetTypes.MLTABLE, path="azureml:odFridgeObjectsTrainingDataset:1",
    mode=InputOutputModes.DIRECT
)

# Validation MLTable with v1 TabularDataset
my_validation_data_input = Input(
    type=AssetTypes.MLTABLE, path="azureml:odFridgeObjectsValidationDataset:1",
    mode=InputOutputModes.DIRECT
)
"""

# 3. コンピュートターゲットの設定

AutoML モデルのトレーニングに使用される[コンピュートターゲット](https://docs.microsoft.com/ja-jp/azure/machine-learning/concept-azure-machine-learning-architecture#computes)を提供する必要があります。画像タスク用の AutoML モデルには、NC、NCv2、NCv3、ND、NDv2、NCasT4 シリーズなどの[GPU SKU](https://docs.microsoft.com/ja-jp/azure/virtual-machines/sizes-gpu)が必要です。より高速なトレーニングのために、NCsv3 シリーズ（v100 GPUを搭載）の使用を推奨します。マルチ GPU VM SKUを備えたコンピュート ターゲットを使用すると、複数の GPU を活用してトレーニング速度を向上させることができます。さらに、複数のノードを備えたコンピュート ターゲットを設定すると、モデルのハイパーパラメータをチューニングする際に並列性を活用して、より速くモデルトレーニングを行うことができます。

In [ ]:
from azure.ai.ml.entities import AmlCompute
from azure.core.exceptions import ResourceNotFoundError

compute_name = "gpu-cluster-nc6s"

try:
    _ = ml_client.compute.get(compute_name)
    print("Found existing compute target.")
except ResourceNotFoundError:
    print("Creating a new compute target...")
    compute_config = AmlCompute(
        name=compute_name,
        type="amlcompute",
        size="Standard_NC6s_v3",
        idle_time_before_scale_down=120,
        min_instances=0,
        max_instances=4,
    )
    ml_client.begin_create_or_update(compute_config).result()

# 4. オブジェクト検出トレーニングのための AutoML の設定と実行

AutoML を使用すると、画像データに対して画像分類、オブジェクト検出、セグメンテーションのモデルを簡単にトレーニングすることができます。使用するモデルアルゴリズムとハイパーパラメータを制御したり、手動で指定したハイパーパラメータ空間上でのスイープを実行したり、システムによって自動的にハイパーパラメータスイープを行わせることができます。

## 4.1. モデルの自動ハイパーパラメータスイープ（AutoMode）

AutoML for Imagesを使用する際、最適なモデルを見つけるために自動ハイパーパラメータスイープを実行することができます（この機能をAutoModeと呼びます）。最初のベースラインモデルを取得するには、最初に自AutoModeで実施することをお勧めします。AutoMLにより、モデルアーキテクチャや学習率（learning_rate）、エポック数（number_of_epochs）、トレーニングバッチサイズ（training_batch_size）などの値が決定され、複数パターン自動で試行されます。ハイパーパラメータの検索空間、サンプリング方法、早期終了ポリシーを指定する必要はありません。多くのデータセットに対して、10から20回の実行でうまく機能するでしょう。

AutoModeは、`max_trials`を1より大きい値に設定し、ハイパーパラメータ空間、サンプリング方法、終了ポリシーを省略することでトリガーされます。

以下の関数は、自動スイープのためのAutoMLジョブを設定します：
### image_object_detection() 関数のパラメータ:
`image_object_detection()`関数を使用すると、トレーニングジョブを設定できます。

- `compute` - AutoMLジョブが実行されるコンピュート。この例では、ワークスペースに存在する'gpu-cluster'というコンピュートを使用しています。ワークスペース内の他のコンピュートに置き換えることができます。
- `experiment_name` - 実験の名前。実験は、論理的な機械学習実験に関連する複数の実行を含むAzure MLワークスペース内のフォルダのようなものです。
- `name` - ジョブ/実行の名前。これはオプションのプロパティです。指定されていない場合、ランダムな名前が生成されます。
- `primary_metric` - AutoMLがモデル選択のために最適化するメトリック。
- `target_column_name` - 予測の対象となる列の名前。常に指定する必要があります。このパラメータは'training_data'および'validation_data'に適用されます。
- `training_data` - トレーニングに使用されるデータ。トレーニング特徴列とターゲット列の両方を含むべきです。オプションで、このデータはバリデーションまたはテストデータセットを分離するために分割することができます。
ワークスペースで登録されたMLTableを'<mltable_name>:<version>'の形式で使用することも、ローカルファイルやフォルダをMLTableとして使用することもできます。例：Input(mltable='my_mltable:1')またはInput(mltable=MLTable(local_path="./data"))
`training_data`パラメータは常に提供される必要があります。

### set_limits() 関数のパラメータ:
これは、タイムアウトなどの制限パラメータを設定するためのオプションの設定方法です。

- `timeout_minutes` - AutoMLジョブのタイムアウト時間（分）です。指定されていない場合、ジョブの総タイムアウトのデフォルトは6日間（8,640分）です。
- `max_trials` - スイープする最大の設定数。1から1000の間の整数でなければなりません。特定のモデルアルゴリズムのデフォルトハイパーパラメータのみを探索する場合、このパラメータを1に設定します。デフォルト値は1です。
- `max_concurrent_trials` - 同時に実行できる最大実行数。指定されていない場合、すべての実行が並行して開始されます。指定される場合、1から100の間の整数でなければなりません。デフォルト値は1です。
    注：同時実行数は、指定されたコンピュートターゲットで利用可能なリソースによって制限されます。要件に基づいて、コンピュートターゲットに十分なリソースがあることを確認してください。

In [ ]:
# general job parameters
exp_name = "dpv2-image-object-detection-experiment"

In [ ]:
# Create the AutoML job with the related factory-function.

image_object_detection_job = automl.image_object_detection(
    compute=compute_name,
    experiment_name=exp_name,
    training_data=my_training_data_input,
    validation_data=my_validation_data_input,
    target_column_name="label",
    primary_metric="mean_average_precision",
    tags={"my_custom_tag": "My custom value"},
)

image_object_detection_job.set_limits(
    max_trials=10,
    max_concurrent_trials=2,
)

### Computer Vision 用の AutoML Job を Submit する
ジョブの設定が完了したら、トレーニングデータセットを使ってビジョンモデルをトレーニングするために、ジョブを送信します。

In [ ]:
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(
    image_object_detection_job
)  # submit the job to the backend

print(f"Created job: {returned_job}")

In [ ]:
ml_client.jobs.stream(returned_job.name)

## 4.2. 【オプション】個別の実行

AutoMode が要件に合わない場合、モデルアルゴリズムを探索するために個別の実行を行うことができます。各アルゴリズムに対して適切なデフォルトのハイパーパラメータを提供しています。また、同じモデルアルゴリズムに対して異なるハイパーパラメータの組み合わせで個別の実行を行うこともできます。モデルアルゴリズムは`model_name`パラメータを使用して指定されます。サポートされているモデルアルゴリズムのリストについては、[ドキュメント](https://docs.microsoft.com/ja-jp/azure/machine-learning/how-to-auto-train-image-models?tabs=CLI-v2#configure-model-algorithms-and-hyperparameters)を参照してください。

以下の関数を使用して、個別の実行のためのAutoMLジョブを設定できます：
### set_training_parameters() 関数のパラメータ:
これは、パラメータ空間のスイープ中に変更されない固定設定やパラメータを設定するためのオプションの設定方法です。この関数の主なパラメータには以下が含まれます：

- `model_name` - トレーニングジョブで使用したいMLアルゴリズムの名前。サポートされているモデルアルゴリズムについては、この[ドキュメント](https://docs.microsoft.com/ja-jp/azure/machine-learning/how-to-auto-train-image-models?tabs=CLI-v2#supported-model-algorithms)を参照してください。
- `number_of_epochs` - トレーニングのエポック数。正の整数でなければなりません（デフォルト値は15）。
- `layers_to_freeze` - 転移学習のために凍結するモデルのレイヤー数。正の整数でなければなりません（デフォルト値は0）。
- `early_stopping` - トレーニング中に早期終了ロジックを有効にします。ブール値でなければなりません（デフォルトはTrue）。
- `optimizer` - トレーニングで使用するオプティマイザーのタイプ。sgd、adam、adamwのいずれかでなければなりません（デフォルトはsgd）。
- `distributed` - 計算ターゲットが複数のGPUを含む場合に分散トレーニングを有効にします。ブール値でなければなりません（デフォルトはTrue）。

特定のアルゴリズム（例えば`yolov5`）のデフォルトのハイパーパラメータ値を使用したい場合、AutoML Image実行のジョブを次のように指定できます：

In [ ]:
# Create the AutoML job with the related factory-function.

image_object_detection_job = automl.image_object_detection(
    compute=compute_name,
    experiment_name=exp_name,
    training_data=my_training_data_input,
    validation_data=my_validation_data_input,
    target_column_name="label",
)

# Set limits
image_object_detection_job.set_limits(timeout_minutes=60)

# Pass the fixed settings or parameters
image_object_detection_job.set_training_parameters(model_name="yolov5")

In [ ]:
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(image_object_detection_job)

print(f"Created job: {returned_job}")

In [ ]:
ml_client.jobs.stream(returned_job.name)

### 4.2.1 MMDetectionのモデルを使用した個別実行（プレビュー）

AutoMLがネイティブにサポートするモデルに加えて、オブジェクト検出をサポートするMMDetectionバージョン2.28.2の任意のモデルを使用して個別実行を行うことができます。使用可能なモデルのリストについては、この[ドキュメント](https://github.com/open-mmlab/mmdetection/blob/v2.28.2/docs/en/model_zoo.md)を参照してください。

Azure Machine Learning ではモデルを管理するマネージドレジストリがあり、そこから厳選された MMDetection のモデルを利用できます。これらのモデルには、合理的なデフォルトのハイパーパラメーターを提供しています。厳選されたモデルのリストを以下のコードスニペットを使用して取得することができます。

In [ ]:
registry_ml_client = MLClient(credential, registry_name="azureml")

models = registry_ml_client.models.list()
object_detection_models = []
for model in models:
    try:
        model = registry_ml_client.models.get(model.name, label="latest")
        if model.tags.get("task", "") == "object-detection":
            object_detection_models.append(model.name)
    except Exception as ex:
        print(f"Error while accessing registry model list: {ex}")

object_detection_models

あるモデル（例えば、`vfnet_r50_fpn_mdconv_c3-c5_mstrain_2x_coco`）を試したい場合、AutoML Imageの実行ジョブを次のように指定します：

In [ ]:
# Create the AutoML job with the related factory-function.

image_object_detection_job = automl.image_object_detection(
    compute=compute_name,
    experiment_name=exp_name,
    training_data=my_training_data_input,
    validation_data=my_validation_data_input,
    target_column_name="label",
)

# Set limits
image_object_detection_job.set_limits(timeout_minutes=60)

# Pass the fixed settings or parameters
image_object_detection_job.set_training_parameters(
    model_name="vfnet_r50_fpn_mdconv_c3-c5_mstrain_2x_coco"
)

In [ ]:
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(image_object_detection_job)

print(f"Created job: {returned_job}")

In [ ]:
ml_client.jobs.stream(returned_job.name)

## 4.3. モデルの手動ハイパーパラメーター探索

AutoML for Computer Vision を使用する際には、AutoML により定義されたパラメータ空間を超えて、利用者の設定でハイパーパラメーター探索を行い、最適なモデルを見つけることもできます。この例では、COCOという、20万枚以上のラベル付き画像と80以上のラベルカテゴリーを含む大規模なオブジェクト検出、セグメンテーション、キャプションデータセットに事前学習された`yolov5`および`fasterrcnn_resnet50_fpn`モデルのハイパーパラメーターを探索します。学習率（learning_rate）、最適化アルゴリズム（optimizer）、学習率スケジューラ（lr_scheduler）など、さまざまな値の範囲から選択し、最適な'mean_average_precision'を持つモデルを生成します。ハイパーパラメーターの値が指定されていない場合は、指定されたアルゴリズムのデフォルト値が使用されます。

set_sweep関数は、探索設定を構成するために使用されます：
### set_sweep() パラメーター:
- `sampling_algorithm` - 定義されたパラメータ空間を探索するために使用するサンプリング方法。サポートされているサンプリング方法のリストについては、この[ドキュメント](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-auto-train-image-models?tabs=SDK-v2#sampling-methods-for-the-sweep)を参照してください。
- `early_termination` - 性能が低い実行を早期に終了させるポリシー。終了ポリシーが指定されていない場合、すべての設定は完了するまで実行されます。サポートされている早期終了ポリシーについては、この[ドキュメント](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-auto-train-image-models?tabs=SDK-v2#early-termination-policies)を参照してください。

このパラメータ空間からランダムサンプリングを使用してサンプルを選択し、これらの異なるサンプルで合計10回のイテレーションを試み、計算ターゲットで一度に2回のイテレーションを実行します。パラメータ空間が持つパラメータが多いほど、最適なモデルを見つけるためにはより多くのイテレーションが必要であることに注意してください。

Bandit policy を利用し、パフォーマンスが低い設定（最も良いパフォーマンスの設定から20%のスラック内にないもの）を終了させることで、計算リソースを大幅に節約します。

モデルとハイパーパラメーター探索の詳細については、[ドキュメント](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters)を参照してください。

In [ ]:
# Create the AutoML job with the related factory-function.

image_object_detection_job = automl.image_object_detection(
    compute=compute_name,
    experiment_name=exp_name,
    training_data=my_training_data_input,
    validation_data=my_validation_data_input,
    target_column_name="label",
    primary_metric=ObjectDetectionPrimaryMetrics.MEAN_AVERAGE_PRECISION,
    tags={"my_custom_tag": "My custom value"},
)

In [ ]:
# Set limits
image_object_detection_job.set_limits(
    timeout_minutes=60,
    max_trials=10,
    max_concurrent_trials=2,
)

In [ ]:
# Pass the fixed settings or parameters
image_object_detection_job.set_training_parameters(
    early_stopping=True, evaluation_frequency=1
)

In [ ]:
# Configure sweep settings
image_object_detection_job.set_sweep(
    sampling_algorithm="random",
    early_termination=BanditPolicy(
        evaluation_interval=2, slack_factor=0.2, delay_evaluation=6
    ),
)

In [ ]:
# Define search space
image_object_detection_job.extend_search_space(
    [
        SearchSpace(
            model_name=Choice(["yolov5"]),
            learning_rate=Uniform(0.0001, 0.01),
            model_size=Choice(["small", "medium"]),  # model-specific
            # image_size=Choice(640, 704, 768),  # model-specific; might need GPU with large memory
        ),
        SearchSpace(
            model_name=Choice(["fasterrcnn_resnet50_fpn"]),
            learning_rate=Uniform(0.0001, 0.001),
            optimizer=Choice(["sgd", "adam", "adamw"]),
            min_size=Choice([600, 800]),  # model-specific
            # warmup_cosine_lr_warmup_epochs=Choice([0, 3]),
        ),
    ]
)

In [ ]:
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(
    image_object_detection_job
)  # submit the job to the backend

print(f"Created job: {returned_job}")

In [ ]:
ml_client.jobs.stream(returned_job.name)

ハイパーパラメーター探索を行う際には、HyperDrive UI を使用して試行されたさまざまな設定を視覚化することが役立ちます。この UI にアクセスするには、上記のメイン automl イメージジョブのUI内の「Child jobs」タブに移動します。これが HyperDrive の「Parent Run」です。次に、HyperDrive の「Parent Run」の「Trials」タブに移動します。あるいは、こちらから直接 HyperDrive の「Parent Run」を表示し、「Trials」タブに移動することもできます：

In [ ]:
hd_job = ml_client.jobs.get(returned_job.name + "_HD")
hd_job

### 4.3.1 MMDetection のモデルを使用した手動ハイパーパラメーター探索（プレビュー）

個別実行で MMDetection バージョン2.28.2の任意のモデルを使用できるのと同様に、これらのモデルを使用してハイパーパラメーター探索を行うこともできます。[AutoML](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-auto-train-image-models?tabs=CLI-v2#configure-model-algorithms-and-hyperparameters)にネイティブにサポートされているモデルと[MMDetection](https://github.com/open-mmlab/mmdetection/blob/v2.28.2/docs/en/model_zoo.md)からのモデルの組み合わせを選択することもできます。

この例では、`deformable_detr_twostage_refine_r50_16x2_50e_coco`、`sparse_rcnn_r50_fpn_300_proposals_crop_mstrain_480-800_3x_coco`、`yolov5`のモデルを対象に、学習率（learning_rate）、モデルサイズ（model_size）など、一連の値から選択して、最適な'MeanAveragePrecision'を持つモデルを生成します。

In [ ]:
# Create the AutoML job with the related factory-function.

image_object_detection_job = automl.image_object_detection(
    compute=compute_name,
    experiment_name=exp_name,
    training_data=my_training_data_input,
    validation_data=my_validation_data_input,
    target_column_name="label",
    primary_metric=ObjectDetectionPrimaryMetrics.MEAN_AVERAGE_PRECISION,
    tags={"my_custom_tag": "My custom value"},
)

# Set limits
image_object_detection_job.set_limits(
    timeout_minutes=240,
    max_trials=10,
    max_concurrent_trials=2,
)

# Configure sweep settings
image_object_detection_job.set_sweep(
    sampling_algorithm="random",
    early_termination=BanditPolicy(
        evaluation_interval=2, slack_factor=0.2, delay_evaluation=6
    ),
)

# Define search space
image_object_detection_job.extend_search_space(
    [
        SearchSpace(
            model_name=Choice(["yolov5"]),
            learning_rate=Uniform(0.0001, 0.01),
            model_size=Choice(["small", "medium"]),  # model-specific
        ),
        SearchSpace(
            model_name=Choice(
                [
                    "deformable_detr_twostage_refine_r50_16x2_50e_coco",
                    "sparse_rcnn_r50_fpn_300_proposals_crop_mstrain_480-800_3x_coco",
                ]
            ),
            learning_rate=Uniform(0.00001, 0.0001),
            number_of_epochs=Choice([15, 20]),
        ),
    ]
)

In [ ]:
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(
    image_object_detection_job
)  # submit the job to the backend

print(f"Created job: {returned_job}")

In [ ]:
ml_client.jobs.stream(returned_job.name)

# 5. 最良モデルの取得
MLFLowClient を使用して、完了した AutoML トライアルの結果（モデル、成果物、メトリクスなど）にアクセスします。

Azure Machine Learning ワークスペースは、MLflow と互換性があります。つまり、追加の構成なしで MLflow サーバーとして機能できます。 各ワークスペースには、MLflow によってワークスペースに接続するために使用される MLflow 追跡 URI があります。 Azure Machine Learning ワークスペースは、MLflow と連携するように既に構成されているため、追加の構成は不要です。

## MLFlow クライアントの初期化

AutoML によって作成されたモデルと成果物には MLFlow のインターフェースからアクセスできます。

ここで MLFlow クライアントを初期化し、MLFlow クライアント経由でバックエンドを Azure ML に設定します。


重要: MLFlow の最新ライブラリをインストールする必要があります。

    pip install azureml-mlflow

    pip install mlflow

In [ ]:
!/anaconda/envs/azureml_py310_sdkv2/bin/pip install azureml-mlflow
!/anaconda/envs/azureml_py310_sdkv2/bin/pip install mlflow

### MLFlow の tracking URIを取得する

In [ ]:
import mlflow

# Obtain the tracking URL from MLClient
MLFLOW_TRACKING_URI = ml_client.workspaces.get(
    name=ml_client.workspace_name
).mlflow_tracking_uri

print(MLFLOW_TRACKING_URI)

In [ ]:
# Set the MLFLOW TRACKING URI
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
print(f"\nCurrent tracking uri: {mlflow.get_tracking_uri()}")

In [ ]:
from mlflow.tracking.client import MlflowClient

# Initialize MLFlow client
mlflow_client = MlflowClient()

### AutoML parent Job を取得する

In [ ]:
job_name = returned_job.name

# Example if providing an specific Job name/ID
# job_name = "salmon_camel_5sdf05xvb3"

# Get the parent run
mlflow_parent_run = mlflow_client.get_run(job_name)

print("Parent Run: ")
print(mlflow_parent_run)

In [ ]:
# Print parent run tags. 'automl_best_child_run_id' tag should be there.
print(mlflow_parent_run.data.tags)

### AutoML の最良のジョブ（試行）を取得する

In [ ]:
# Get the best model's child run

best_child_run_id = mlflow_parent_run.data.tags["automl_best_child_run_id"]
print(f"Found best child run id: {best_child_run_id}")

best_run = mlflow_client.get_run(best_child_run_id)

print("Best child run: ")
print(best_run)

## 最良モデルのメトリクスを取得する
実行したAutoMLのメトリクスにアクセスします。

In [ ]:
import pandas as pd

pd.DataFrame(best_run.data.metrics, index=[0]).T

## 最良モデルをダウンロード
実行したAutoMLの結果（モデル、成果物、メトリクスなど）にアクセスします。

In [ ]:
# Create local folder
local_dir = "./artifact_downloads"
if not os.path.exists(local_dir):
    os.mkdir(local_dir)

In [ ]:
# Download run's artifacts/outputs
local_path = mlflow_client.download_artifacts(
    best_run.info.run_id, "outputs", local_dir
)
print(f"Artifacts downloaded in: {local_path}")
print(f"Artifacts: {os.listdir(local_path)}")

In [ ]:
import os

mlflow_model_dir = os.path.join(local_dir, "outputs", "mlflow-model")

# Show the contents of the MLFlow model folder
os.listdir(mlflow_model_dir)

# You should see a list of files such as the following:
# ['artifacts', 'conda.yaml', 'MLmodel', 'python_env.yaml', 'python_model.pkl', 'requirements.txt']

# 6. 最良モデルを登録・デプロイする

## 6.1 マネージドオンラインエンドポイントを作成
マネージドオンラインエンドポイントを作成します。マネージドオンラインエンドポイントはリアルタイムの推論リクエストの処理をサーバレスで提供します。


In [ ]:
# import required libraries
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
    ProbeSettings,
)

In [ ]:
# Creating a unique endpoint name with current datetime to avoid conflicts
import datetime

online_endpoint_name = "od-fridge-items-" + datetime.datetime.now().strftime(
    "%m%d%H%M%f"
)

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is a sample online endpoint for deploying model",
    auth_mode="key",
    tags={"foo": "bar"},
)
print(online_endpoint_name)

In [ ]:
ml_client.begin_create_or_update(endpoint).result()

## 6.2 最良モデルを登録・デプロイ

### Register model

In [ ]:
model_name = "od-fridge-items-mlflow-model"
model = Model(
    path=f"azureml://jobs/{best_run.info.run_id}/outputs/artifacts/outputs/mlflow-model/",
    name=model_name,
    description="my sample object detection model",
    type=AssetTypes.MLFLOW_MODEL,
)

# for downloaded file
# model = Model(
#     path=mlflow_model_dir,
#     name=model_name,
#     description="my sample object detection model",
#     type=AssetTypes.MLFLOW_MODEL,
# )

registered_model = ml_client.models.create_or_update(model)

In [ ]:
registered_model.id

### デプロイ

In [ ]:
from azure.ai.ml.entities import OnlineRequestSettings

# Setting the request timeout to 90 seconds. Please note that if you use a GPU compute, inference would be faster
# and this setting may not be required.
req_timeout = OnlineRequestSettings(request_timeout_ms=90000)

In [ ]:
deployment = ManagedOnlineDeployment(
    name="od-fridge-items-mlflow-deploy",
    endpoint_name=online_endpoint_name,
    model=registered_model.id,
    instance_type="Standard_DS4_V2",
    instance_count=1,
    request_settings=req_timeout,
    liveness_probe=ProbeSettings(
        failure_threshold=30,
        success_threshold=1,
        timeout=2,
        period=10,
        initial_delay=2000,
    ),
    readiness_probe=ProbeSettings(
        failure_threshold=10,
        success_threshold=1,
        timeout=10,
        period=10,
        initial_delay=2000,
    ),
)

In [ ]:
ml_client.online_deployments.begin_create_or_update(deployment).result()

In [ ]:
# od fridge items deployment to take 100% traffic
endpoint.traffic = {"od-fridge-items-mlflow-deploy": 100}
ml_client.begin_create_or_update(endpoint).result()

### エンドポイントの詳細情報を取得

In [ ]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

### Test the deployment

In [ ]:
# Create request json
import base64

sample_image = os.path.join(dataset_dir, "images", "1.jpg")


def read_image(image_path):
    with open(image_path, "rb") as f:
        return f.read()


request_json = {
    "input_data": {
        "columns": ["image"],
        "data": [base64.encodebytes(read_image(sample_image)).decode("utf-8")],
    }
}

In [ ]:
import json

request_file_name = "sample_request_data.json"

with open(request_file_name, "w") as request_file:
    json.dump(request_json, request_file)

In [ ]:
resp = ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name=deployment.name,
    request_file=request_file_name,
)

## Visualize detections
Now that we have scored a test image, we can visualize the bounding boxes for this image.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from PIL import Image
import numpy as np
import json

IMAGE_SIZE = (18, 12)
plt.figure(figsize=IMAGE_SIZE)
img_np = mpimg.imread(sample_image)
img = Image.fromarray(img_np.astype("uint8"), "RGB")
x, y = img.size

fig, ax = plt.subplots(1, figsize=(15, 15))
# Display the image
ax.imshow(img_np)

# draw box and label for each detection
detections = json.loads(resp)
for detect in detections[0]["boxes"]:
    label = detect["label"]
    box = detect["box"]
    conf_score = detect["score"]
    if conf_score > 0.6:
        ymin, xmin, ymax, xmax = (
            box["topY"],
            box["topX"],
            box["bottomY"],
            box["bottomX"],
        )
        topleft_x, topleft_y = x * xmin, y * ymin
        width, height = x * (xmax - xmin), y * (ymax - ymin)
        print(
            f"{detect['label']}: [{round(topleft_x, 3)}, {round(topleft_y, 3)}, "
            f"{round(width, 3)}, {round(height, 3)}], {round(conf_score, 3)}"
        )

        color = np.random.rand(3)  #'red'
        rect = patches.Rectangle(
            (topleft_x, topleft_y),
            width,
            height,
            linewidth=3,
            edgecolor=color,
            facecolor="none",
        )
        ax.add_patch(rect)
        plt.text(topleft_x, topleft_y - 10, label, color=color, fontsize=20)
plt.show()

### デプロイとエンドポイントを削除

In [ ]:
ml_client.online_endpoints.begin_delete(name=online_endpoint_name)

# 次のステップ: 最適なモデルをロードして予測を試す

モデルをローカルにロードすることは、ノートブックをモデルと互換性のある環境で実行していることを前提としています。モデルが期待する依存関係のリストは、AutoMLによって生成されたMLFlowモデル（mlflow-modelフォルダ内の'conda.yaml'ファイル）に指定されています。

AutoMLモデルが異なる環境でリモートでトレーニングされ、現在このノートブックを実行しているローカルのconda環境とは異なる依存関係を持っているため、モデルをロードしたい場合はいくつかのオプションがあります：

1. モデルをメモリ内にローカルでロードして予測を試す推奨される方法は、MLFlowモデルのフォルダ内のconda.yamlファイルに指定されている依存関係で新しい/クリーンなconda環境を作成し、その後MLFlowを使用してモデルをロードし、この同じフォルダ内のノートブック**mlflow-model-local-inference-test.ipynb**に説明されているように.predict()を呼び出すことです。

2. Azure ML SDKおよびAutoMLの使用に使用した現在のconda環境に、conda.yamlに指定されているすべてのパッケージ/依存関係をインストールすることもできます。MLflow SDKには、現在の環境に依存関係をインストールする方法もあります。しかし、このオプションは現在の環境にインストールされている内容によっては、パッケージバージョンの競合のリスクがあります。

3. また、次のコマンドを使用することもできます：mlflow models serve -m 'xxxxxxx'

# Next Steps
AutoML で、Regression, Image-Classification, NLP-Text-Classification, Time-Series-Forcasting のようなそのほかのタスクを試してみることも可能です。